##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 사용자 정의 학습: 기초

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/eager/custom_training"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/eager/custom_training.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/eager/custom_training.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

이전 튜토리얼에서 우리는 머신러닝을 위한 기초 빌딩 블록인 자동미분(automatic differentiation)을 위한 텐서플로우 API들을 알아보았습니다. 이번 튜토리얼에서는 이전 튜토리얼에서 소개되었던 초기 타입의 텐서플로우를 사용하여 간단한 머신러닝을 구축해보겠습니다. 

텐서플로우는 상용구를 줄이기 위해 유용한 추상화를 제공하는 고수준 신경망 API인 (`tf.keras`)를 포함하고 있습니다. 신경망에 관련하여 일을 하고 있는 사람들에게는 이러한 고수준의 API들을 강하게 추천합니다. 그러나 이번 짧은 튜토리얼에서는 탄탄한 기초를 기르기 위한 신경망 학습을 다루겠습니다. 

## 설정

In [2]:
import tensorflow as tf

tf.enable_eager_execution()

C:\Users\82108\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 변수

텐서플로우 안에서 텐서(Tensor)는 상태가 없고, 변경이 불가능한(immutable stateless) 객체입니다. 그러나 머신러닝은 상태가 변경될 필요가 있습니다(stateful). 예를 들어, 모델 학습에서 예측을 계산하기 위한 동일한 코드는 시간이 지남에 따라 다른 양상(희망적으로, 더 낮은 손실로 가는 방향으로)을 보여야 합니다. 이 계산 과정을 통해 변화되어야 하는 상태를 표현하기 위해 상태가 변경 가능한 파이썬 언어에 의존한 선택이 가능합니다. 

In [ ]:
# 파이썬 state 사용
x = tf.zeros([10, 10])
x += 2  # 이것은 x = x + 2 같으며, 초기값 x를 변경하지 않습니다.
print(x)

그러나 텐서플로우는 상태가 변경 가능한 연산자들이 내장되어 있으며, 이 연산자들은 상태를 표현하기 위한 저수준 파이썬 표현보다 사용하기가 더 좋습니다. 예를 들어, 모델에서 가중치를 나타내기 위해서 텐서플로우 변수를 사용하는것이 편하고 효율적입니다.  

텐서플로우 변수는 값을 저장하고 텐서플로우 계산에 사용될 때 묵시적으로 저장된 값을 읽어오는 객체입니다. `tf.assign_sub`, `tf.scatter_update` 등은 텐서플로우 변수에 저장되있는 값을 조작하는 연산자들입니다.

In [ ]:
v = tf.Variable(1.0)
assert v.numpy() == 1.0

# 값 재배열
v.assign(3.0)
assert v.numpy() == 3.0

# 텐서플로우 연산자 내에서 `v` 사용 
v.assign(tf.square(v))
assert v.numpy() == 9.0

변수들을 사용한 계산은 그래디언트가 계산될 때 자동적으로 추적됩니다. 임베딩(embedding)을 나타내는 변수의 경우 초기값으로부터 드물게 업데이트됩니다. 이는 계산과 메모리에 있어 더욱 효율적입니다. 

또한 변수를 사용하는 것은 코드를 읽는 과정에서 변경 가능한 상태(state mutable)의 조각을 빠르게 인식하는 방법입니다.

## 예: 선형모델 피팅

몇가지 개념들을 설명해보겠습니다. 우리는 지금까지 간단한 모델을 구축하고 학습시키기 위해 ---`Tensor`, `GradientTape`, `Variable` --- 등을 사용하였습니다. 이것은 전형적으로 다음의 과정을 포함합니다.

1. 모델 정의
2. 손실함수 정의
3. 훈련 데이터 가져오기
4. 훈련 데이터를 통한 실행, 데이터에 최적화하기 위한 "옵티마이저(optimizer)" 사용한 변수 조정

이번 튜토리얼에서는 선형모델의 간단한 예제를 살펴보겠습니다. `f(x) = x * W + b`, 위 모델은 `W` 와 `b` 두 변수를 가지고 있는 선형모델이며, 잘 학습된 모델이 `W = 3.0` and `b = 2.0`의 값을 갖도록 데이터를 합성할 것입니다.

### 모델 정의

변수들과 계산을 요약하기 위한 간단한 클래스를 정의해봅시다.

In [3]:
class Model(object):
  def __init__(self):
    # 변수 초기화 (5.0, 0.0)
    # 실제로는 임의의 값으로 초기화 되어야합니다.
    self.W = tf.Variable(5.0)
    self.b = tf.Variable(0.0)
    
  def __call__(self, x):
    return self.W * x + self.b
  
model = Model()

assert model(3.0).numpy() == 15.0

### 손실 함수 정의

손실 함수는 주어진 입력에 대한 모델의 출력이 원하는 출력과 얼마나 잘 일치하는지를 측정합니다. L2 규제항(regularization)을 적용한 손실 함수를 사용하겠습니다.

In [4]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

### 훈련 데이터 얻기

약간의 잡음과 훈련 데이터를 합칩니다.

In [5]:
TRUE_W = 3.0
TRUE_b = 2.0
NUM_EXAMPLES = 1000

inputs  = tf.random_normal(shape=[NUM_EXAMPLES])
noise   = tf.random_normal(shape=[NUM_EXAMPLES])
outputs = inputs * TRUE_W + TRUE_b + noise

모델을 훈련시키기 전에, 모델의 현재 상태를 시각화합시다. 모델의 예측을 빨간색으로, 훈련데이터를 파란색으로 구성합니다.

In [6]:
import matplotlib.pyplot as plt

plt.scatter(inputs, outputs, c='b')
plt.scatter(inputs, model(inputs), c='r')
plt.show()

print('Current loss: '),
print(loss(model(inputs), outputs).numpy())

<Figure size 640x480 with 1 Axes>

Current loss: 
9.401943


### 훈련 루프 정의

현재 우리는 네트워크와 훈련 데이터를 가지고 있습니다. 모델의 변수(`W` 와 `b`)를 업데이트하기 위해 훈련 데이터를 사용하여 훈련시킵니다. 그리고 [경사하강(gradient descent)](https://en.wikipedia.org/wiki/Gradient_descent)를 사용하여 손실을 감소시킵니다. 경사하강에는 여러가지 방법이 있으며, `tf.train.Optimizer` 에 구현되어있습니다. 이러한 구현을 사용하는것을 강력히 추천드립니다. 그러나 이번 튜토리얼에서는 기본적인 방법을 사용하겠습니다.

In [ ]:
def train(model, inputs, outputs, learning_rate):
  with tf.GradientTape() as t:
    current_loss = loss(model(inputs), outputs)
  dW, db = t.gradient(current_loss, [model.W, model.b])
  model.W.assign_sub(learning_rate * dW)
  model.b.assign_sub(learning_rate * db)

마지막으로, 훈련 데이터를 반복적으로 실행하고, `W` 와 `b`의 변화과정을 확인합니다.

In [ ]:
model = Model()

# 도식화를 위해 W값과 b값들의 변화를 저장합니다.
Ws, bs = [], []
epochs = range(10)
for epoch in epochs:
  Ws.append(model.W.numpy())
  bs.append(model.b.numpy())
  current_loss = loss(model(inputs), outputs)

  train(model, inputs, outputs, learning_rate=0.1)
  print('Epoch %2d: W=%1.2f b=%1.2f, loss=%2.5f' %
        (epoch, Ws[-1], bs[-1], current_loss))

# Let's plot it all
plt.plot(epochs, Ws, 'r',
         epochs, bs, 'b')
plt.plot([TRUE_W] * len(epochs), 'r--',
         [TRUE_b] * len(epochs), 'b--')
plt.legend(['W', 'b', 'true W', 'true_b'])
plt.show()
  

## 다음 단계

이번 튜토리얼에서는 `Variable`를 다루었으며, 지금까지 논의된 초기 타입의 텐서플로우를 사용하여 간단한 선형모델을 구축하고 훈련시켰습니다.

이론적으로, 이것은 머신러닝 연구에 텐서플로우를 사용하는데 필요한 대부분입니다. 실제로, 신경망에 있어 `tf.keras`와 고수준 API들은 고수준 빌딩 블록("layer"로 불리는)을 제공하고, 저장 및 복원을 위한 유틸리티, 손실함수 모음, 최적화 전략 모음 등을 제공하기 때문에 더욱 편리합니다. 